In [2]:
import dislib as dl
from dislib.utils import shuffle
from dislib.math import svd

ModuleNotFoundError: No module named 'dislib'

In [1]:
gate1 = dl.random_array(shape=(20,20,5,3), block_size=(250, 250))

NameError: name 'dl' is not defined

In [90]:
gate1.shape

(2000, 2000)

In [91]:
import cProfile
import re
cProfile.run('[u,d,v] = svd(gate1)')

         353990 function calls (272922 primitive calls) in 21.451 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(argsort)
       32    0.000    0.000    0.006    0.000 <__array_function__ internals>:2(atleast_2d)
      971    0.003    0.000    0.838    0.001 <__array_function__ internals>:2(block)
       96    0.000    0.000    0.045    0.000 <__array_function__ internals>:2(concatenate)
      168    0.000    0.000    0.194    0.001 <__array_function__ internals>:2(dot)
       64    0.000    0.000    0.002    0.000 <__array_function__ internals>:2(intersect1d)
      184    0.001    0.000    0.215    0.001 <__array_function__ internals>:2(norm)
      848    0.002    0.000    0.011    0.000 <__array_function__ internals>:2(result_type)
      168    0.001    0.000    0.629    0.004 <__array_function__ internals>:2(sum)
      122    0.000    0.000  

In [83]:
import numpy as np

In [84]:
gate2 = np.random.rand(2000,2000)

In [85]:
cProfile.run('[u,d,v] = np.linalg.svd(gate2)')

         1674 function calls in 3.964 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    3.961    3.961 <__array_function__ internals>:2(svd)
        1    0.002    0.002    3.964    3.964 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 _asarray.py:16(asarray)
      820    0.000    0.000    0.000    0.000 api.py:297(compss_delete_object)
      820    0.000    0.000    0.000    0.000 api.py:78(compss_delete_object)
        3    0.000    0.000    0.001    0.000 array.py:72(__del__)
        3    0.000    0.000    0.001    0.000 array.py:74(<listcomp>)
        1    0.000    0.000    0.000    0.000 linalg.py:111(get_linalg_error_extobj)
        1    0.000    0.000    0.000    0.000 linalg.py:116(_makearray)
        2    0.000    0.000    0.000    0.000 linalg.py:121(isComplexType)
        2    0.000    0.000    0.000    0.000 linalg.py:134(_realType)
        1    0.000    0.000    0.

In [7]:
def contraction4x4(gate1, gate2):
    shape1 = gate1.shape
    gate1_contraction = gate1.reshape(shape1[0]*shape1[1]*shape1[2], shape1[3])
    shape2 = gate2.shape
    gate2_contraction = gate2.reshape(shape2[0], shape2[1]*shape2[2]*shape2[3])
    contraction = np.dot(gate1_contraction, gate2_contraction)
    final_form = contraction.reshape(shape1[0], shape1[1], shape1[2], shape2[1], shape2[2], shape2[3])
    return final_form

In [8]:
c_4x4 = contraction4x4(gate1, gate2)
c_4x4.shape

AttributeError: 'Array' object has no attribute 'reshape'

In [6]:
def contraction_vertical(gate1, gate2):
    shape1 = gate1.shape
    gate1_contraction = gate1.reshape(shape1[0]*shape1[1]*shape1[2], shape1[3])
    gate2_contraction = gate2.transpose(1,0,2,3)
    shape2 = gate2.shape
    gate2_contraction = gate2_contraction.reshape(shape2[0], shape2[1]*shape2[2]*shape2[3])
    contraction = np.dot(gate1_contraction, gate2_contraction)
    contraction = contraction.reshape(shape1[0], shape1[1], shape1[2], shape2[1], shape2[2], shape2[3])
    contraction = contraction.transpose(0,3,1,2,4,5)
    shapeC = contraction.shape
    final_form = contraction.reshape(shapeC[0]*shapeC[1], shapeC[2], shapeC[3]*shapeC[4], shapeC[5])
    return final_form

In [7]:
c_vertical = contraction_vertical(gate1, gate2)
c_vertical.shape

(4, 2, 4, 2)

In [8]:
def contraction_horizontal(gate1, gate2):
    gate1_contraction = gate1.transpose(0,1,3,2)
    shape1 = gate1_contraction.shape
    gate1_contraction = gate1_contraction.reshape(shape1[0]*shape1[1]*shape1[2], shape1[3])
    shape2 = gate2.shape
    gate2_contraction = gate2.reshape(shape2[0], shape2[1]*shape2[2]*shape2[3])
    contraction = np.dot(gate1_contraction, gate2_contraction)
    contraction = contraction.reshape(shape1[0], shape1[1], shape1[2], shape2[1], shape2[2], shape2[3])
    contraction = contraction.transpose(0,1,3,4,2,5)
    shapeC = contraction.shape
    final_form = contraction.reshape(shapeC[0], shapeC[1]*shapeC[2], shapeC[3], shapeC[4]*shapeC[5])
    return final_form

In [9]:
c_horizontal = contraction_horizontal(c_vertical, c_vertical)
c_horizontal.shape

(4, 4, 4, 4)

In [10]:
def run_test():
    c = gate1
    for i in range(3):
        c = contraction_vertical(c, c)
        print(c.shape)
        c = contraction_horizontal(c, c)
        print(c.shape)

(4, 2, 4, 2)
(4, 4, 4, 4)
(16, 4, 16, 4)
(16, 16, 16, 16)
(256, 16, 256, 16)
         205 function calls in 0.132 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        6    0.000    0.000    0.060    0.010 <__array_function__ internals>:2(dot)
        1    0.000    0.000    0.132    0.132 <ipython-input-10-76ac5a889385>:1(run_test)
        3    0.000    0.000    0.088    0.029 <ipython-input-6-ccdc64f7ae11>:1(contraction_vertical)
        3    0.000    0.000    0.043    0.014 <ipython-input-8-172de626b5e3>:1(contraction_horizontal)
        1    0.000    0.000    0.132    0.132 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 _exceptions.py:146(__init__)
       11    0.000    0.000    0.001    0.000 iostream.py:197(schedule)
       10    0.000    0.000    0.000    0.000 iostream.py:309(_is_master_process)
       10    0.000    0.000    0.000    0.000 iostream.py:322(_schedule_flush)
       10    0.000    

MemoryError: Unable to allocate 32.0 GiB for an array with shape (65536, 65536) and data type float64

In [ ]:
def truncation_horizontal(gate1, gate2, dim_tol):
    gate1_contraction = gate1.transpose(0,1,3,2)
    shape1 = gate1_contraction.shape
    gate1_contraction = gate1_contraction.reshape(shape1[0]*shape1[1]*shape1[2], shape1[3])
    shape2 = gate2.shape
    gate2_contraction = gate2.reshape(shape2[0], shape2[1]*shape2[2]*shape2[3])
    contraction = np.dot(gate1_contraction, gate2_contraction)
    
    [u,s,v] = np.linalg.svd(contraction)

    u = u[:,:dim_tol]
    v = v[:dim_tol,:]
    
    u_final = u.reshape(shape1[0], shape1[1], shape1[2], dim_tol)
    v_final = v.reshape(dim_tol, shape2[1], shape2[2], shape2[3])
    
    return u_final, v_final

In [ ]:
u, v = truncation_horizontal(gate1, gate2, 2)
print(u.shape)
print(v.shape)

In [ ]:
c = contraction_vertical(gate1, gate2)
print(c.shape)
u, v = truncation_horizontal(c, c, 2)
print(u.shape)
print(v.shape)